In [431]:
# !pip install alpaca-trade-api

In [432]:
import alpaca_trade_api as tradeapi
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, StockLatestQuoteRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
import config
from datetime import datetime, timedelta, timezone
import json
import mplfinance as mpf
import matplotlib.pyplot as plt
import os
import pandas as pd
import polars as pl
import plotly.graph_objects as go
import seaborn as sns
import time


In [433]:
POSITION_FILE: str = 'position.json' 
SYMBOL: str = 'SPY'
MAX_TRADES_PER_DAY: int = 10        # Maximum trades allowed per day
DAILY_LOSS_LIMIT: int  = 100       # Maximum daily loss allowed (in dollars)
POSITION_SIZE_PCT: float = 0.1        # 10% of buying power per trade
COOLDOWN: int = 60 * 5              # 5 minutes cooldown in seconds
TRAILING_STOP_PCT: float = 0.02       # 2% trailing stop

# Initial position dictionary
initial_position_data = {
    "side": "long",
    "entry_price": 180.50,
    "qty": 10,
    "high": 180.50,
    "low": 180.00,
    "entry_time": datetime.now().isoformat(),
    "last_exit": ""
} 

# # Simulated activities
# activities = [{
#         "symbol": SYMBOL,
#         "side": "buy",
#         "transaction_time": datetime.now() - timedelta(hours=2),
#         "realized_pl": 0.00},
#     {
#         "symbol": SYMBOL,
#         "side": "sell",
#         "transaction_time": datetime.now() - timedelta(hours=1, minutes=30),
#         "realized_pl": 15.75},
#     {
#         "symbol": SYMBOL,
#         "side": "buy",
#         "transaction_time": datetime.now() - timedelta(minutes=45),
#         "realized_pl": 0.00},
#     {
#         "symbol": SYMBOL,
#         "side": "sell",
#         "transaction_time": datetime.now() - timedelta(minutes=30),
#         "realized_pl": -5.25}]

In [434]:
def create_position_file_if_not_exists():
    """Creates the position.json file with initial data if it doesn't exist."""
    if not os.path.exists(POSITION_FILE):
        try:
            with open(POSITION_FILE, 'w') as f:
                json.dump(initial_position_data, f, indent=4)
            print(f"'{POSITION_FILE}' not found. Created and initialized it.")
        except Exception as e:
            print(f"Error creating '{POSITION_FILE}': {e}")
    else:
        print(f"'{POSITION_FILE}' already exists.")

In [435]:
create_position_file_if_not_exists()

'position.json' not found. Created and initialized it.


In [436]:
def load_position() -> list[dict]:
    """Loads the trading position from the position.json file."""
    # Ensure the file exists before attempting to load
    create_position_file_if_not_exists()
    try:
        with open(POSITION_FILE, 'r') as f:
            position_data = json.load(f)
            print(f"Successfully loaded position from {POSITION_FILE}")
            return position_data
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {POSITION_FILE}. File might be corrupted.")
        # file is corrupted, lets delete it and re-create
        os.remove(POSITION_FILE)
        print(f"Deleted corrupted {POSITION_FILE}. Re-creating it.")
        try:
             with open(POSITION_FILE, 'r') as f:
                 position_data = json.load(f)
                 return position_data # Should now load the initial data
        except:
             return initial_position_data # Fallback if even re-creation fails (unlikely)
    except Exception as e:
        print(f"An unexpected error occurred while reading {POSITION_FILE}: {e}. Returning initial position.")
        return initial_position_data


In [437]:
load_position()

'position.json' already exists.
Successfully loaded position from position.json


{'side': 'long',
 'entry_price': 180.5,
 'qty': 10,
 'high': 180.5,
 'low': 180.0,
 'entry_time': '2025-06-13T16:04:02.153733',
 'last_exit': ''}

In [438]:
def close_position():
    """Closes the current trading position."""
    try:
        # api.close_position(SYMBOL)
        print("Position closed.")
        # reset_position()
    except Exception as e:
        print(f"Close error: {e}")

In [439]:
def save_position(data):
    """Saves the current trading position to the position.json file."""
    POSITION_FILE = 'position.json'
    with open(POSITION_FILE, 'w') as f:
        json.dump(data, f)
    print("Position saved")

In [440]:
def reset_position():
    """Resets the position by deleting the position.json file."""
    POSITION_FILE = 'position.json'
    if os.path.exists(POSITION_FILE):
        os.remove(POSITION_FILE)
    print("Positioned reset")

In [441]:
reset_position()

Positioned reset


In [442]:
# writte a function that connects tto Alpaca API and returns the api object
def connect_to_alpaca() -> tradeapi.REST | None:
    """Connects to the Alpaca API and returns the API object."""
    api_key = os.getenv("ALPACA_API_KEY")
    api_secret = os.getenv("ALPACA_SECRET_KEY")
    api_url = os.getenv("ALPACA_BASE_URL", "https://paper-api.alpaca.markets")

    if not api_key or not api_secret:
        print("API key and secret must be set in environment variables.")
        return None
    
    try:
        api = tradeapi.REST(
            key_id=api_key,
            secret_key=api_secret,
            base_url=api_url
        )
        print("Connected to Alpaca API")
        return api
    except Exception as e:
        print(f"Error connecting to Alpaca API: {e}")
        return None


In [443]:
api = connect_to_alpaca()
if api is None:
    print("Failed to connect to Alpaca API. Exiting.")
    exit(1)
# get account information
account = api.get_account()
print(f"Account status: {account.status}")
print(f"Account equity: {account.equity}")
print(f"Account buying power: {account.buying_power}")

Connected to Alpaca API
Account status: ACTIVE
Account equity: 99797.58
Account buying power: 359913.39


In [444]:
def fetch_positions(api) -> list:
    """Fetches and returns the current open positions."""
    try:
        positions = api.list_positions()
        if positions:
            for position in positions:
                print(f"Symbol: {position.symbol}, Qty: {position.qty}, Side: {position.side}, Market Value: {position.market_value}")
        else:
            print("No open positions.")
        return positions
    except Exception as e:
        print(f"Error fetching positions: {e}")
        return []

In [445]:
positions = fetch_positions(api)

Symbol: SPY, Qty: 56.459157247, Side: long, Market Value: 33706.116876


In [446]:
def fetch_activities(api, symbol='SPY', after=None) -> list:
    """Fetches and returns account activities for a specific symbol."""
    try:
        activities = api.get_activities(activity_types=['FILL'], after=after)
        symbol_activities = [activity for activity in activities if activity.symbol == symbol]
        if symbol_activities:
            for activity in symbol_activities:
                realized_pl = getattr(activity, "realized_pl", None)
        else:
            print(f"No activities found for symbol {symbol}.")
        return symbol_activities
    except Exception as e:
        print(f"Error fetching activities: {e}")
        return []

In [447]:
# Fetch activities
activities = fetch_activities(api, after=(datetime.now() - timedelta(days=5)).isoformat() + 'Z')
if activities:
    for activity in activities:
        time_str = activity.transaction_time.strftime('%H:%M:%S')
        print(
            f"Recent activity: {activity.side} "
            f"{activity.qty} shares of {activity.symbol} "
            f"at {activity.price} "
            f"on {time_str}"
        )
else:
    print("No recent activities.")

Recent activity: buy 1 shares of SPY at 597.95 on 18:15:26
Recent activity: sell 0.01 shares of SPY at 601.032 on 15:10:01
Recent activity: buy 5 shares of SPY at 601.09 on 15:09:41
Recent activity: buy 1 shares of SPY at 601.15 on 15:09:18
Recent activity: buy 0.001829801 shares of SPY at 601.158 on 15:09:13
Recent activity: buy 0.5 shares of SPY at 601.19 on 15:09:11
Recent activity: buy 1 shares of SPY at 601.19 on 15:09:11
Recent activity: buy 2 shares of SPY at 601.19 on 15:09:11
Recent activity: buy 2 shares of SPY at 601.21 on 15:09:10
Recent activity: sell 0.01 shares of SPY at 600.982 on 15:00:51
Recent activity: buy 1 shares of SPY at 601.02 on 15:00:25
Recent activity: buy 1 shares of SPY at 601.01 on 15:00:25
Recent activity: buy 3 shares of SPY at 601.01 on 15:00:24
Recent activity: buy 1 shares of SPY at 600.97 on 15:00:20
Recent activity: buy 0.00183038 shares of SPY at 600.968 on 15:00:15
Recent activity: buy 0.5 shares of SPY at 601.01 on 15:00:12
Recent activity: buy 

In [448]:
def fetch_open_orders(api, symbol='SPY') -> list:
    """Fetches and returns open orders for a specific symbol."""
    try:
        orders = api.list_orders(status='open', symbols=[symbol])
        if orders:
            for order in orders:
                print(f"Order ID: {order.id}, Symbol: {order.symbol}, Qty: {order.qty}, Side: {order.side}, Type: {order.type}, Status: {order.status}")
        else:
            print(f"No open orders for symbol {symbol}.")
        return orders
    except Exception as e:
        print(f"Error fetching open orders: {e}")
        return []


In [449]:
open_orders = fetch_open_orders(api)

Order ID: 77d32c00-7a59-41e4-b8cc-6eb75bb58b04, Symbol: SPY, Qty: 1, Side: sell, Type: limit, Status: accepted
Order ID: 29ca5027-19f6-41d7-a7d6-86333ba77af6, Symbol: SPY, Qty: 1, Side: sell, Type: limit, Status: accepted
Order ID: c9753f73-44af-4dcb-8ddf-7e8feb75c545, Symbol: SPY, Qty: 1, Side: sell, Type: limit, Status: accepted
Order ID: 4a004081-8eb3-4c35-80e4-216fe660a21b, Symbol: SPY, Qty: 1, Side: sell, Type: limit, Status: accepted
Order ID: d70d5c36-ae3f-4d2b-afb4-b04ecdddd890, Symbol: SPY, Qty: 1, Side: sell, Type: limit, Status: accepted
Order ID: 4a93c88d-a950-4a29-a31d-c0d464508c3c, Symbol: SPY, Qty: 1, Side: sell, Type: limit, Status: new
Order ID: 22f9a46d-2daa-4fa1-a7b7-4292a936217c, Symbol: SPY, Qty: 1, Side: sell, Type: limit, Status: new
Order ID: 555c6275-58fa-4198-bbda-c9528b4d41fa, Symbol: SPY, Qty: 1, Side: sell, Type: limit, Status: new
Order ID: 369cbc94-1b11-4e5c-a2ce-ad75953d8741, Symbol: SPY, Qty: 1, Side: sell, Type: limit, Status: new
Order ID: 343d60be-a8

In [450]:
def fetch_market_clock(api):
    """Fetches and prints the current market clock information."""
    try:
        clock = api.get_clock()
        print(f"Market is currently {'open' if clock.is_open else 'closed'}. Next open: {clock.next_open}, Next close: {clock.next_close}")
        return clock
    except Exception as e:
        print(f"Error fetching market clock: {e}")
        return None

In [451]:
fetch_market_clock(api)

Market is currently closed. Next open: 2025-06-16 09:30:00-04:00, Next close: 2025-06-16 16:00:00-04:00


Clock({   'is_open': False,
    'next_close': '2025-06-16T16:00:00-04:00',
    'next_open': '2025-06-16T09:30:00-04:00',
    'timestamp': '2025-06-13T17:04:02.688954873-04:00'})

In [452]:
def fetch_market_calendar(api, start_date, end_date) -> list:
    """Fetches and returns the market calendar between specified dates."""
    try:
        calendar = api.get_calendar(start=start_date, end=end_date)
        if calendar:
            for day in calendar:
                print(f"Date: {day.date}, Open: {day.open}, Close: {day.close}")
        else:
            print("No market calendar data found for the specified dates.")
        return calendar
    except Exception as e:
        print(f"Error fetching market calendar: {e}")
        return []

In [453]:
fetch_market_calendar(api, start_date=datetime.now().date(), end_date=(datetime.now() + timedelta(days=7)).date())

Date: 2025-06-13 00:00:00, Open: 09:30:00, Close: 16:00:00
Date: 2025-06-16 00:00:00, Open: 09:30:00, Close: 16:00:00
Date: 2025-06-17 00:00:00, Open: 09:30:00, Close: 16:00:00
Date: 2025-06-18 00:00:00, Open: 09:30:00, Close: 16:00:00
Date: 2025-06-20 00:00:00, Open: 09:30:00, Close: 16:00:00


[Calendar({   'close': '16:00',
     'date': '2025-06-13',
     'open': '09:30',
     'session_close': '2000',
     'session_open': '0400',
     'settlement_date': '2025-06-16'}),
 Calendar({   'close': '16:00',
     'date': '2025-06-16',
     'open': '09:30',
     'session_close': '2000',
     'session_open': '0400',
     'settlement_date': '2025-06-17'}),
 Calendar({   'close': '16:00',
     'date': '2025-06-17',
     'open': '09:30',
     'session_close': '2000',
     'session_open': '0400',
     'settlement_date': '2025-06-18'}),
 Calendar({   'close': '16:00',
     'date': '2025-06-18',
     'open': '09:30',
     'session_close': '2000',
     'session_open': '0400',
     'settlement_date': '2025-06-20'}),
 Calendar({   'close': '16:00',
     'date': '2025-06-20',
     'open': '09:30',
     'session_close': '2000',
     'session_open': '0400',
     'settlement_date': '2025-06-23'})]

In [454]:
def get_historical_data_client() -> StockHistoricalDataClient:
    """Creates and returns a StockHistoricalDataClient instance."""
    api_key = os.getenv("ALPACA_API_KEY")
    api_secret = os.getenv("ALPACA_SECRET_KEY")
    if api_key and api_secret:
        return StockHistoricalDataClient(api_key, api_secret)
    else:
        raise ValueError("API key and secret must be provided.")

In [455]:
def build_stock_bars_request(symbol: str, start: datetime, end: datetime, timeframe: TimeFrame) -> StockBarsRequest:
    """Builds and returns a StockBarsRequest object."""
    return StockBarsRequest(
        symbol_or_symbols=[symbol],
        start=start,
        end=end,
        timeframe=timeframe,
        feed='iex'
    )
    if feed:
        params['feed'] = feed
    return StockBarsRequest(**params)

In [456]:
def fetch_historical_data(symbol, start, end, timeframe) -> pd.DataFrame | None:
    """Fetches historical stock data from Alpaca and returns a Polars DataFrame."""
    try:
        client = get_historical_data_client()
        request_params = build_stock_bars_request(symbol, start, end, timeframe)
        bars = client.get_stock_bars(request_params)
        df = bars.df
        if df.empty:
            print("No historical data returned.")
            return None
        df.reset_index(inplace=True)  # Make sure 'timestamp' is a column
        # Set 'timestamp' as the index before converting to polars
        df = df.set_index('timestamp')
        df = df.sort_index()
        return df
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return None


In [457]:
fetch_historical_data('SPY', datetime.now() - timedelta(days=5), datetime.now(), TimeFrame.Day)

,symbol,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2025-06-09 04:00:00+00:00,SPY,599.750,601.250,598.520,599.43,745570.0,9529.0,600.025070
2025-06-10 04:00:00+00:00,SPY,600.335,603.465,599.150,603.10,954803.0,12227.0,601.680572
2025-06-11 04:00:00+00:00,SPY,604.180,605.040,599.275,601.59,1048022.0,14418.0,602.365390
2025-06-12 04:00:00+00:00,SPY,600.020,603.720,599.570,603.72,824021.0,13745.0,602.553171
2025-06-13 04:00:00+00:00,SPY,598.540,601.840,595.520,596.78,1120853.0,14485.0,598.389526


In [458]:
def create_trade_log_file():
    """
    Creates a trade_log.csv file with headers in the same directory as the notebook.
    """
    trade_log_file = 'trade_log.csv'
    headers = "entry_time,exit_time,side,entry_price,exit_price,pnl,logged_at\n"
    if not os.path.exists(trade_log_file):
        with open(trade_log_file, 'w') as f:
            f.write(headers)
        print(f"Created {trade_log_file} with headers.")
    else:
        print(f"{trade_log_file} already exists.")

In [459]:
def log_trade(entry_time, exit_time, side, entry_price, exit_price, pnl):
    """Logs trade details to a CSV file."""
    TRADE_LOG_FILE = 'trade_log.csv'
    headers = ["entry_time", "exit_time", "side", "entry_price", "exit_price", "pnl", "logged_at"]
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    file_exists = os.path.exists(TRADE_LOG_FILE)

    with open(TRADE_LOG_FILE, 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(headers)
        writer.writerow([entry_time, exit_time, side, entry_price, exit_price, pnl, timestamp])
    print("Trade logged.")


In [460]:
def place_order(symbol, side, qty, price):
    try:
        api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type='limit',
        limit_price=round(price, 2),
        time_in_force='gtc')
        print(f"{side.upper()} order placed at {price}")
    except Exception as e:
        print(f"Order error: {e}")

In [461]:
# Use the existing pandas DataFrame 'df' for the latest price and order placement
# This is a test to sell 1 unit of 'SPY' at the latest price.
df = fetch_historical_data(SYMBOL, datetime.now() - timedelta(days=1), datetime.now(), TimeFrame.Minute)
if df is not None and not df.empty:
    latest_price = float(df['close'].iloc[-1])
    print(f"Latest price for {SYMBOL}: {latest_price}")
    place_order(SYMBOL, 'sell', 1, latest_price)
else:
    print("No recent bars found.")

Latest price for SPY: 601.01
SELL order placed at 601.01


In [462]:
def filtered_daily_trade(activities) -> list:
    """Filters trades based on their side."""
    filtered_activities = [activity for activity in activities if getattr(activity, 'side', None) in ['buy', 'sell']]
    
    # Filter trades for today only
    today = datetime.now().date()
    filtered_trades = [
        trade for trade in filtered_activities
        if pd.to_datetime(getattr(trade, 'transaction_time')).date() == today
    ]
    return filtered_trades


In [463]:
def check_daily_limits() -> tuple[int, float]:
    """Checks and returns the number of trades and total PNL for today."""
    print("Checking daily limits...")
    try:
        # Use midnight UTC for today
        
        # Use midnight UTC for today, formatted as ISO 8601 with 'Z'
        after = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0).isoformat().replace('+00:00', 'Z')
        activities = fetch_activities(api, after=after)
        print(f"Activity: {activities}")
        daily_trades = filtered_daily_trade(activities)
        print(f"Today's trades: {daily_trades}")
    except Exception as e:
        print(f"Error fetching activities: {e}")
        return 0, 0

    # Use attribute access for realized_pl
    today_pnl = sum(float(getattr(trade, "realized_pl", 0) or 0) for trade in daily_trades)
    print(f"Today's total PNL: {today_pnl}")

    try:
        trade_count = len([t for t in daily_trades if getattr(t, 'side', None) in ['buy', 'sell']])
        print(f"Trade count: {trade_count}")
        print(f"Today's total PNL: {today_pnl}")
        print(f"returning: {trade_count, today_pnl}")
        return trade_count, today_pnl
    except RuntimeError as e:
        print(f"{e}")
        return 0, 0

In [464]:
check_daily_limits()

Checking daily limits...
Activity: [AccountActivity({   'activity_type': 'FILL',
    'cum_qty': '1',
    'id': '20250613141526465::bb74f2e3-7306-4a87-9a02-812305c5b9e8',
    'leaves_qty': '0',
    'order_id': '41c8d603-2eaf-4834-af71-118282b72a81',
    'order_status': 'filled',
    'price': '597.95',
    'qty': '1',
    'side': 'buy',
    'symbol': 'SPY',
    'transaction_time': '2025-06-13T18:15:26.465747Z',
    'type': 'fill'})]
Today's trades: [AccountActivity({   'activity_type': 'FILL',
    'cum_qty': '1',
    'id': '20250613141526465::bb74f2e3-7306-4a87-9a02-812305c5b9e8',
    'leaves_qty': '0',
    'order_id': '41c8d603-2eaf-4834-af71-118282b72a81',
    'order_status': 'filled',
    'price': '597.95',
    'qty': '1',
    'side': 'buy',
    'symbol': 'SPY',
    'transaction_time': '2025-06-13T18:15:26.465747Z',
    'type': 'fill'})]
Today's total PNL: 0.0
Trade count: 1
Today's total PNL: 0.0
returning: (1, 0.0)


(1, 0.0)

In [ ]:
def get_data(symbol, timeframe='1m', limit=30) -> pd.DataFrame:
    """Fetches historical stock data and returns a DataFrame."""
    try:

        # Compute time range
        end = datetime.now(timezone.utc)
        start = end - timedelta(minutes=limit)

        # Choose correct timeframe
        if timeframe == '1m':
            tf = TimeFrame(1, TimeFrameUnit.Minute)
        elif timeframe == '5m':
            tf = TimeFrame(5, TimeFrameUnit.Minute)
        else:
            raise ValueError(f"Unsupported timeframe: {timeframe}")
        # Initialize client
        client = get_historical_data_client()

        # Build request parameters
        request_params = build_stock_bars_request(symbol, start, end, tf)

        # Fetch and process data
        bars = client.get_stock_bars(request_params).df
        df = bars.reset_index().tail(limit)
        df.set_index('timestamp', inplace=True)
        return df

    except Exception as e:
        print(f"Data error: {e}")
        return pd.DataFrame()


In [473]:
df = get_data(SYMBOL, '1m', 30)
df

""


In [474]:
# Interactive chart with plotly
def create_candlestick_chart(df):
    # Create candlestick trace
    candlestick = go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['high'],
        low=df['low'],
        close=df['close']
    )

    # Create figure and add trace
    fig = go.Figure(data=[candlestick])

    # Update layout
    fig.update_layout(
        title=f'{SYMBOL} Interactive Candlestick Chart',
        xaxis_title='Time',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False # Hide range slider
    )

    # Show the plot
    fig.show()

In [475]:
create_candlestick_chart(df)

KeyError: 'Open'

In [ ]:
def calculate_ema(data, period=9) -> pd.Series:
    """Calculates the Exponential Moving Average (EMA) for the given data."""
    
    if data.empty:
        raise ValueError("Input data is empty.")
    ema = data['close'].ewm(span=period, adjust=False).mean()
    return ema

In [ ]:
ema_series = calculate_ema(df, period=12)
ema_series.head()

In [ ]:
ema_series = calculate_ema(df, period=12)
plt.figure(figsize=(12, 5))
sns.lineplot(x=df.index, y=df['close'], label='Close Price')
sns.lineplot(x=df.index, y=ema_series, label='EMA (12)', color='orange')
plt.title('Close Price and EMA (12)')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
def calculate_rsi(data, period=14) -> pd.Series:
    """Calculates the Relative Strength Index (RSI) for the given data."""
    if data.empty:
        raise ValueError("Input data is empty.")
    delta = data['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [ ]:
rsi_series = calculate_rsi(df, period=14)
rsi_series

In [ ]:
rsi_series = calculate_rsi(df, period=14)
plt.figure(figsize=(12, 4))
sns.lineplot(x=df.index, y=rsi_series, label='RSI (14)', color='purple')
plt.axhline(70, color='red', linestyle='--', label='Overbought (70)')
plt.axhline(30, color='green', linestyle='--', label='Oversold (30)')
plt.title('RSI (14)')
plt.xlabel('Time')
plt.ylabel('RSI Value')
plt.legend()
plt.show()

In [ ]:
def average_volume(data, window=20) -> pd.Series:
    """Calculates the average volume over a specified window."""
    if data.empty:
        raise ValueError("Input data is empty.")
    
    avg_vol = data['volume'].rolling(window=window).mean()
    return avg_vol

In [ ]:
average_volume_series = average_volume(df)
average_volume_series

In [ ]:
average_volume_series = average_volume(df)
plt.figure(figsize=(12, 4))
sns.lineplot(x=df.index, y=df['volume'], label='Volume')
sns.lineplot(x=df.index, y=average_volume_series, label='Average Volume (20)', color='orange')
plt.title('Volume and 20-Period Average Volume')
plt.xlabel('Time')
plt.ylabel('Volume')
plt.legend()
plt.show()

In [ ]:
def calculate_slope(series, window=5) -> pd.Series:
    """Calculates the slope of the given series over a specified window."""
    if series.empty:
        raise ValueError("Input series is empty.")
    
    slope = series.diff().rolling(window=window).mean()
    return slope

In [ ]:
cal_slope_series = calculate_slope(df['close'])
cal_slope_series

In [ ]:
cal_slope_series = calculate_slope(df['close'])
plt.figure(figsize=(12, 4))
sns.lineplot(x=df.index, y=cal_slope_series, label='Slope (5)', color='blue')
plt.title('Slope of Close Price (5-Period Rolling)')
plt.xlabel('Time')
plt.ylabel('Slope')
plt.legend()
plt.show()

In [ ]:
def check_missing_values(df) -> pd.Series:
    """Checks for missing values in the DataFrame and returns a Series with counts."""
    if df.empty:
        raise ValueError("Input DataFrame is empty.")

    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0]
    print("Missing values in each column:\n", missing_values)
    return missing_values

In [ ]:
check_missing_values(df)

In [ ]:
def remove_dividends_splits(df, columns) -> pd.DataFrame:
    """Removes specified columns from the DataFrame."""
    if df.empty:
        raise ValueError("Input DataFrame is empty.")
    try:
        df = df.drop(columns=columns)
    except Exception as e:
        print(f"Data error: {e}")
    print("Remaining columns after removal:\n", df.columns)
    return df

In [ ]:
df_removed = remove_dividends_splits(df, ['dividends', 'splits'])
df_removed.columns

In [ ]:
def save_all_open_positions(api, filename="position.json") -> None:
    """Saves all open positions to a JSON file."""
    
    positions = api.list_positions()

    # Convert each position object to a dictionary
    positions_data = [p._raw for p in positions]
    with open(filename, "w") as f:
        json.dump(positions_data, f, indent=4)
    print(f"Saved {len(positions_data)} open positions to {filename}")

In [ ]:
def fetch_open_positions(api) -> list:
    """Fetches and returns all open positions."""
    try:
        positions = api.list_positions()
        if not positions:
            print("No open positions.")
        return positions
    except Exception as e:
        print(f"Error fetching positions: {e}")
        return []


In [ ]:
save_all_open_positions(api)
open_positions = fetch_open_positions(api)
for position in open_positions:
    symbol = position.symbol
    qty = position.qty
    entry_price = position.avg_entry_price
    current_price = position.current_price
    unrealized_pl = position.unrealized_pl
    print(
        f"Symbol: {symbol}, Qty: {qty}, Entry: {entry_price}, "
        f"Current: {current_price}, Unrealized PnL: {unrealized_pl}"
    )

In [ ]:
def convert_row_to_numeric(row, df):
    """
    Converts the values in the given row to a consistent numeric format for all numeric columns in df.
    Returns the modified row.
    """
    numerical_columns = df.select_dtypes(include='number').columns.tolist()
    for col in numerical_columns:
        try:
            val = row[col]
            if pd.isna(val) or str(val).strip() in ['', 'None', 'nan', 'NaN']:
                raise ValueError(f"Missing or empty value for {col}")
            row[col] = float(val) if not isinstance(val, float) else val
        except Exception as e:
            print(f"Data conversion error for '{col}': {e}")
    return row

In [ ]:
def add_indicators(df):
    """
    Adds EMA(9), slope of EMA(9), RSI(14), and 20-period average volume to the DataFrame.
    Returns the DataFrame with new columns: 'ema9', 'slope9', 'rsi', 'avg_volume'.
    """
    df = df.copy()
    df['ema9'] = calculate_ema(df, period=9)
    df['slope9'] = calculate_slope(df['ema9'])
    df['rsi'] = calculate_rsi(df)
    df['avg_volume'] = average_volume(df)
    return df

In [ ]:
def main():
    

    position = load_position()
    print(f"position: {position}")

    # --- Modified Cooldown Check ---
    now = datetime.now()
    print(f"now: {now}")

    # Check if last_exit is a non-empty string before attempting to parse
    #print(f"position type: {type(position[0])}")
    last_exit_str = position[0].get('last_exit', '') # Use .get to safely access, default to ''
    print(f"position[0]['last_exit'] string: '{last_exit_str}'")

    if last_exit_str:
        try:
            last_exit = datetime.fromisoformat(last_exit_str)
            print(f"last_exit (parsed): {last_exit}")

            time_from_last_exit = int((now - last_exit).total_seconds())
            time_left = COOLDOWN - time_from_last_exit
            print(f"time_from_last_exit: {time_from_last_exit}")
            print(f"time_left: {time_left}")

            if (time_left > 0):
                print(f"Cooldown active for {time_left} more seconds. Last trade was at {last_exit.strftime('%H:%M:%S')}")
                return  # Exit if cooldown is active
        except ValueError:
            print(f"Warning: 'last_exit' value '{last_exit_str}' is not a valid ISO format. No cooldown applied.")
    else:
        print("'last_exit' is empty. No cooldown applied.")

    # Rest of your trade logic continues here if cooldown is not active
    trade_count, daily_pnl = check_daily_limits()
    if trade_count >= MAX_TRADES_PER_DAY or daily_pnl <= -DAILY_LOSS_LIMIT:
        print("Daily limit reached."
              f" Trade count: {trade_count}, Daily PnL: {daily_pnl}. "
              "No trades will be made today."
        )
        return
    
    try: 
        client = get_historical_data_client()
        quote = client.get_stock_latest_quote(StockLatestQuoteRequest(symbol_or_symbols=SYMBOL))
    except ValueError as e:
        print(f"Error initializing data client: {e}")
        return
    

    df_1m = get_data(SYMBOL, '1m', 30)
    df_5m = get_data(SYMBOL, '5m', 30)

    if df_1m.empty or df_5m.empty:
        print("Data unavailable.")
        return

    # Check for missing values
    if not df_1m.isnull().sum().sum() == 0 or not df_5m.isnull().sum().sum() == 0:
        print("Missing data detected. Skipping trade.")
        return

    df_1m = add_indicators(df_1m)
    df_5m = add_indicators(df_5m)

    last = df_1m.iloc[-1].copy()
    #print("Raw last row values:")
    for k, v in last.items():
        print(f"  {k}: {v}")

    last = df_1m.iloc[-1].copy()
    last = convert_row_to_numeric(last, df_1m)

    # #last

    slope = float(df_5m['slope9'].iloc[-1])  # Updated to use 'slope9'
    print(f"Slope: {slope}")
    side = position[0].get('side')
    print(f"Side: {side}")
    qty = position[0].get('qty', 0)
    print(f"Qty: {qty}")
    entry_price = position[0].get('avg_entry_price', 0)
    print(f"Entry price: {entry_price}")
    buying_power = float(api.get_account().cash)
    current_price = quote[SYMBOL].ask_price
    print(f"Current price: {current_price}")
    print(f"Buying power: {buying_power}")
    trade_qty = int((buying_power * POSITION_SIZE_PCT) / last['close'])
    print(f"Trade qty: {trade_qty}")

    # No position
    if not side:
        print("No position found")
        # if last['Close'] > last['Open'] and last['Close'] > last['ema9'] and last['rsi'] < 65 and last['Volume'] > last['avg_volume'] and slope > 0:
        #     price = last['Close'] * 1.001
        #     place_order('buy', trade_qty, price)
        #     save_position({'side': 'long', 'entry_price': price, 'qty': trade_qty, 'high': price, 'entry_time': now.isoformat()})

        # elif last['Close'] < last['Open'] and last['Close'] < last['ema9'] and last['rsi'] > 55 and last['Volume'] > last['avg_volume'] and slope < 0:
        #     price = last['Close'] * 0.999
        #     place_order('sell', trade_qty, price)
        #     save_position({'side': 'short', 'entry_price': price, 'qty': trade_qty, 'low': price, 'entry_time': now.isoformat()})

    # Long Position
    elif side == 'long':
        print("Long Position")
        # print("Long Position")
        # position['high'] = max(position['high'], last['Close'])
        # trail_stop = position['high'] * (1 - TRAILING_STOP_PCT)
        # print(f"[LONG] Comparing last['Close'] = {last['Close']} (type: {type(last['Close'])}) to trail_stop = {trail_stop}")
        # last_close = float(last['Close'])
        # if last_close <= trail_stop or last_close < last['ema9']:
        #     close_position()
        #     exit_price = last['Close']
        #     pnl = (exit_price - entry_price) * qty
        #     log_trade(position['entry_time'], now.isoformat(), side, entry_price, exit_price, pnl)
        #     #reset_position()
        #     save_position({'last_exit': now.isoformat()})

    # # Short Position
    elif side == 'short':
        print("Short Position")
        # position['low'] = min(position['low'], last['Close'])
        # trail_stop = position['low'] * (1 + TRAILING_STOP_PCT)
        # print(f"[SHORT] Comparing last['Close'] = {last['Close']} (type: {type(last['Close'])}) to trail_stop = {trail_stop}")
        # last_close = float(last['Close'])
        # if last_close >= trail_stop or last_close > last['ema9']:
        #     close_position()
        #     exit_price = last['Close']
        #     pnl = (entry_price - exit_price) * qty
        #     log_trade(position['entry_time'], now.isoformat(), side, entry_price, exit_price, pnl)
        #     #reset_position()
        #     save_position({'last_exit': now.isoformat()})


In [ ]:
if __name__ == "__main__":
    create_trade_log_file()
    main()